In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
from Data_Enrichment import get_features

RAW_DIR = "data/raw"
df_feats, feature_cols = get_features(RAW_DIR)
df_feats = df_feats[ (df_feats['season_end_year'] != 2026)]



/workspaces/Machine-learning/Data_Enrichment.py:300: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["ballon_dor_winner"] = df["ballon_dor_winner"].fillna(False).astype(bool)


In [3]:
def corr_var(df_feats):
    df_feats.drop(columns=['clean_sheets', 'nb_in_group', 'matches_played', 'g_per90', 'ga_per90'], inplace=True)
    return df_feats
df = corr_var(df_feats)

In [4]:
# Filtro limpio: deja las variables temporales relevantes + identificación + target
df_model = df.filter(
    regex=r'(player_id|player_name|lag1|delta|_w$|height|main_position|position|season_end_year|ballon_dor_winner|age|age_penalty)'
).copy()


In [5]:
# Entrenamiento: todas las temporadas hasta 2024
train = df_model[df_model['season_end_year'] < 2025]

# Predicción: temporada 2025
test = df_model[df_model['season_end_year'] == 2025]


In [6]:
# Features y target
X_train = train.drop(columns=['ballon_dor_winner'])
y_train = train['ballon_dor_winner']

X_test = test.drop(columns=['ballon_dor_winner'])
y_test = test['ballon_dor_winner']


In [7]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Identificar tipos
num_cols = X_train.select_dtypes(include='number').columns.drop(['season_end_year'], errors='ignore')
cat_cols = ['position', 'main_position']

# Preprocesador
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
], remainder='drop')


In [8]:
from sklearn.ensemble import RandomForestClassifier

rf = Pipeline([
    ('prep', preprocessor),
    ('model', RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        random_state=42,
        class_weight='balanced_subsample'
    ))
])

rf.fit(X_train, y_train)


,steps,"[('prep', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [9]:
from sklearn.metrics import classification_report

y_pred = rf.predict(X_test)
y_proba = rf.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

       False       1.00      1.00      1.00      3503

    accuracy                           1.00      3503
   macro avg       1.00      1.00      1.00      3503
weighted avg       1.00      1.00      1.00      3503



In [10]:
test = test.copy()
test['pred_proba'] = y_proba

top_candidates = (
    test[['player_name', 'season_end_year', 'pred_proba']]
    .sort_values('pred_proba', ascending=False)
)

top_candidates.head(10)


,player_name,season_end_year,pred_proba
40075,Robert Lewandowski (38253),2025,0.043333
65866,Iago Aspas (72047),2025,0.030000
6136,Harry Kane (132098),2025,0.023333
38500,Federico Valverde (369081),2025,0.023333
43756,Bryan Mbeumo (413039),2025,0.016667
16409,Jamie Vardy (197838),2025,0.013333
44223,Erling Haaland (418560),2025,0.010000
56680,Julián Alvarez (576024),2025,0.010000
4517,Hakan Çalhanoğlu (126414),2025,0.006667
11502,Matteo Politano (165895),2025,0.006667


In [11]:
# Extraer el modelo RandomForest ya entrenado
model = rf.named_steps['model']

# Obtener las columnas transformadas del preprocesador
# (OneHotEncoder genera columnas nuevas, así que hay que recuperarlas)
encoder = rf.named_steps['prep'].named_transformers_['cat']
encoded_cols = encoder.get_feature_names_out(['position', 'main_position'])

num_cols_scaled = rf.named_steps['prep'].named_transformers_['num'].feature_names_in_

# Combinar nombres de todas las features finales del pipeline
all_features = np.concatenate([num_cols_scaled, encoded_cols])

# Importancias del modelo
importances = model.feature_importances_

# Crear DataFrame ordenado
feat_imp = pd.DataFrame({
    'feature': all_features,
    'importance': importances
}).sort_values('importance', ascending=False)

feat_imp.head(20)


,feature,importance
8,a_per90_z_lag1,0.153470
20,matches_played_z_lag1,0.140673
4,ga_per90_z_lag1,0.097148
16,pen_share_z_lag1,0.075730
6,g_per90_z_lag1,0.071807
7,g_per90_z_delta,0.067441
24,a_per90_w,0.042674
5,ga_per90_z_delta,0.040773
22,ga_per90_w,0.038439
23,g_per90_w,0.035635


In [12]:
a = feat_imp["importance"].sum()
print(r"the 20 most important features explain ", round((feat_imp.head(20)["importance"].sum()/a)*100,2), "% of the model")

the 20 most important features explain  94.19 % of the model


In [13]:
# Top 20 important feature names
top_features = feat_imp.head(20)['feature'].values
top_features



array(['a_per90_z_lag1', 'matches_played_z_lag1', 'ga_per90_z_lag1',
       'pen_share_z_lag1', 'g_per90_z_lag1', 'g_per90_z_delta',
       'a_per90_w', 'ga_per90_z_delta', 'ga_per90_w', 'g_per90_w',
       'a_per90_z_delta', 'pen_share_w', 'minutes_played_z_delta',
       'minutes_played_z_lag1', 'pen_share_z_delta',
       'main_position_Attack', 'age', 'height', 'discipline_rate_z_lag1',
       'position_Attack - Right Winger'], dtype=object)